In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.font_manager as font_manager
from matplotlib import rc
import f90nml

rc('text', usetex=True)
rc('font', family='serif')
thin = 1
fs = 14
plt.rcParams["font.size"] = 12

def read_control(experiment):
    namelist = f90nml.read(
        '../data/' + experiment + '/' + experiment + '.out')
    tmp = namelist['nml_parameter']
    N_log_scale = tmp['N_log_scale']
    M_log_scale = tmp['M_log_scale']
    N_time = tmp['N_time']
    NN = tmp['NN']
    NM = tmp['NM']
    NR = tmp['NR']
    NE = tmp['NE']
    N_min = tmp['N_min']
    N_max = tmp['N_max']
    M_min = tmp['M_min']
    M_max = tmp['M_max']
    Ro_min = tmp['Ro_min']
    Ro_max = tmp['Ro_max']
    e_min = tmp['e_min']
    e_max = tmp['e_max']
    figure_directory = "../figure/" + experiment
    if not os.path.isdir(figure_directory):
        os.makedirs(figure_directory)
    return (N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max,
            M_min, M_max, Ro_min, Ro_max, e_min, e_max, figure_directory)

In [ ]:
experiment = 'floquet_NN200'
(N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
 Ro_min, Ro_max,e_min, e_max, figure_directory) = read_control(experiment)

log_N_axis = np.linspace(np.log(N_min), np.log(N_max), NN, endpoint=True)
N_axis = np.exp(log_N_axis)
log_M_axis = np.linspace(np.log(M_min), np.log(M_max), NM, endpoint=True)
M_axis = np.exp(log_M_axis)
Ro_axis = np.linspace(Ro_min, Ro_max, NR, endpoint=True)
e_axis = np.linspace(e_min, e_max, NE, endpoint=True)

# IN = 1
c_levels = np.linspace(0, 0.04, 30)
thin = 2

Ro = 0.95
ir = np.where(np.isclose(Ro_axis, Ro))[0][0]
e_array = [0.1, 0.2, 0.4, 0.6, 0.8, 0.9]
colors = ['r', 'm', 'c', 'g', 'b', 'purple']

shape = np.array([NE, NM])
size = shape.prod()
data_type = np.dtype('<f8')
data_bytes = round(int(data_type.name[-2:]) / 8)
offset = size * data_bytes * ir

growth_rate_N = np.zeros([6, NM, NN])
growth_rate_N_max = np.zeros([6, NN])

for IN in range(NN):
    file_open = open(
        '../data/' + experiment + '/growth_rate_R{:0>6}.out'.format(IN), 'r')
    data_tmp = np.fromfile(
        file_open, dtype=data_type, count=size, offset=offset)
    growth_rate_e = data_tmp.reshape(shape, order='F')
    growth_rate_e[np.isclose(growth_rate_e, 0, atol=1.e-4)] = 0
    for j in range(6):
        e = e_array[j]
        ie = np.where(np.isclose(e_axis, e))[0][0]
        growth_rate_N[j, :, IN] = growth_rate_e[ie, :]
    print(IN)

# del growth_rate

fig = plt.figure(figsize=[6, 7])
fig.subplots_adjust(left=0.12, bottom=0.08, right=0.78,
                    top=0.95, wspace=0.2, hspace=0.25)

ax1 = fig.add_subplot(211)
cont1 = ax1.contourf(N_axis, M_axis[::thin], 
                     growth_rate_N[3, ::thin, :],
                     levels=c_levels, cmap=cm.jet, extend="max")
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_xlim(1.e0, 1.e2)
ax1.set_ylim(1.e0, 1.e3)
ax1.set_ylabel(r'$\tan \phi$', fontsize=fs)
ax1.set_title('(a)', fontsize=fs, loc='left');
ax1.set_xticks([1, 10, 100])
ax1.set_xlabel(r'$N / f$', fontsize=fs)
cax = fig.add_axes((0.815, 0.564, 0.02, 0.388))
cb = fig.colorbar(cont1, ticks=[0, 0.01, 0.02, 0.03, 0.04],
                  orientation='vertical', ax=ax1, cax=cax,
                  label=r"$\lambda/f$");
cax = cb.ax
text = cax.yaxis.label
font = font_manager.FontProperties(size=fs)
text.set_font_properties(font)

ax2 = fig.add_subplot(212)
for j in range(6):
    ax2.plot(N_axis, np.max(growth_rate_N[j, :, :], 0),
             c=colors[j], label=r'$e = {}$'.format(e_array[j]));

grid_y_ticks_minor = np.arange(0, 0.041, 0.001)
grid_y_ticks_major = np.arange(0, 0.05, 0.01)

ax2.set_yticks(grid_y_ticks_minor, minor=True)
ax2.set_yticks(grid_y_ticks_major, major=True)

ax2.grid(zorder=10)
ax2.grid(which='minor', axis='x')
ax2.grid(which='major', axis='y')
ax2.set_xlim(1.e0, 1.e2)
ax2.set_ylim(0, 4.e-2);
ax2.set_xscale('log')
ax2.grid()
ax2.set_title('(b)', fontsize=fs, loc='left');
ax2.set_xlabel(r'$N / f$', fontsize=fs)
ax2.set_ylabel(r'$\lambda_{\rm max}/f$', fontsize=fs)
ax2.legend(bbox_to_anchor=(1.02, 1.0), loc='upper left');

figure_name = figure_directory + '/' +  'floquet_against_N'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png');
